In [7]:
import pandas as pd

# File paths for the datasets
fedex_file_path = 'Fedex 15-07.xlsx'
# updated_sku_data_final_path = 'Updated_SKU_Data_Final.xlsx'

# Load the FedEx data
fedex_data = pd.read_excel(fedex_file_path, sheet_name='Sheet1', skiprows=1)

In [8]:
import pandas as pd

# Load the Excel file
file_path = 'Updated_SKU_Data_Final_Updated.xlsx'
new_updated_sku_data = pd.read_excel(file_path)

# Define the conditions
condition_true_weight = new_updated_sku_data['Variant Weight'] > new_updated_sku_data['Variant Metafield: custom.volumetric_weight [single_line_text_field]']
condition_volumetric = new_updated_sku_data['Variant Metafield: custom.volumetric_weight [single_line_text_field]'] > new_updated_sku_data['Variant Weight']

# Update the 'Weight Type' column based on the conditions
new_updated_sku_data.loc[condition_true_weight, 'Weight Type'] = 'True Weight'
new_updated_sku_data.loc[condition_volumetric, 'Weight Type'] = 'Volumetric'

new_updated_sku_data


,Variant SKU,Status,Variant ID,Variant Weight,Variant Weight Unit,Variant Price,Variant Cost,Variant Metafield: custom.height [single_line_text_field],Variant Metafield: custom.width [single_line_text_field],Variant Metafield: custom.length [single_line_text_field],Variant Metafield: custom.weight [single_line_text_field],Variant Metafield: custom.volume [single_line_text_field],Variant Metafield: custom.volumetric_weight [single_line_text_field],Shipping price nl,Exceed Weight Quantity,Exceed Length Quantity,Exceed Width,Exceed Height,Exceed Girth,Weight Type
0,TR8PRO-R2R-BUN2-EU,Active,42514735399106,126.6000,kg,4671.00,NaN,0.0,0.0,0.0,0.0,0.0,186.0,141.314448,0,0,1,0,1,Volumetric
1,TR8PRO-R2R-BUN1-EU,Active,42514407063746,126.6000,kg,3901.00,NaN,0.0,0.0,0.0,0.0,0.0,144.0,109.480800,0,0,0,0,1,Volumetric
2,TR80L-1-TK,Active,47223492411737,110.0000,kg,8478.00,NaN,0.0,0.0,0.0,0.0,0.0,117.0,89.016312,0,1,0,0,1,Volumetric
3,TR8PRO-1-TK-EU,Active,47223428612441,40.3209,kg,6241.00,NaN,0.0,0.0,0.0,0.0,0.0,111.0,84.468648,0,0,0,0,1,Volumetric
4,SA-09 BUNDLE,Active,47582883807577,8.0000,kg,338.00,NaN,0.0,0.0,0.0,16.5,0.0,84.0,64.004160,0,0,0,0,1,Volumetric
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,DRKV08346,Draft,48338871779673,0.4000,kg,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,True Weight
616,TR8-08-B-PART1,Draft,48373033795929,0.0000,kg,30.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,True Weight
617,SP-24-BLUE-TRX,Draft,48522933436761,0.5000,kg,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,True Weight
618,HE-SPUBP20,Draft,49006423966041,0.0000,kg,92.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,True Weight


### OLD Operation

In [14]:
import pandas as pd

fedex_data = fedex_data.rename(columns={
    fedex_data.columns[0]: 'Weight',
    fedex_data.columns[2]: 'Price'
})[['Weight', 'Price']]

# Remove any row where 'Price' is NaN
fedex_data.dropna(subset=['Price'], inplace=True)

# Load the Updated SKU Data
updated_sku_data = new_updated_sku_data

# Function to calculate Base Shipment
def calculate_base_shipment(row):
    if row['Weight Type'] == 'Volumetric':
        return row['Variant Metafield: custom.volumetric_weight [single_line_text_field]']
    elif row['Weight Type'] == 'True Weight':
        return row['Variant Weight']
    else:
        return None

# Apply the function to create the Base Shipment column
updated_sku_data['Base Shipment'] = updated_sku_data.apply(calculate_base_shipment, axis=1)

# Function to find the closest price based on the base shipment weight
def find_closest_price(base_shipment_weight):
    closest_row = fedex_data.iloc[(fedex_data['Weight'] - base_shipment_weight).abs().argsort()[:1]]
    return closest_row['Price'].values[0] if not closest_row.empty else None

# Apply the function to calculate the Shipping Price
updated_sku_data['Shipping Price Netherlands'] = updated_sku_data['Base Shipment'].apply(find_closest_price)

# Display the first few rows of the final dataset
updated_sku_data.head(), fedex_data.head()


(          Variant SKU  Status      Variant ID  Variant Weight  \
 0  TR8PRO-R2R-BUN2-EU  Active  42514735399106        126.6000   
 1  TR8PRO-R2R-BUN1-EU  Active  42514407063746        126.6000   
 2          TR80L-1-TK  Active  47223492411737        110.0000   
 3      TR8PRO-1-TK-EU  Active  47223428612441         40.3209   
 4        SA-09 BUNDLE  Active  47582883807577          8.0000   
 
   Variant Weight Unit  Variant Price  Variant Cost  \
 0                  kg         4671.0           NaN   
 1                  kg         3901.0           NaN   
 2                  kg         8478.0           NaN   
 3                  kg         6241.0           NaN   
 4                  kg          338.0           NaN   
 
    Variant Metafield: custom.height [single_line_text_field]  \
 0                                                0.0           
 1                                                0.0           
 2                                                0.0           
 3        

In [15]:
def calculate_additional_charges(row):
    if row['Exceed Length Quantity'] == 1 or row['Exceed Weight Quantity'] == 1 or row['Exceed Width'] == 1 or row['Exceed Height'] == 1 or row['Exceed Girth'] == 1:
        return 18
    else:
        return 0
    
# Apply the function to create a new column for the additional charges
updated_sku_data['Additional Charges'] = updated_sku_data.apply(calculate_additional_charges, axis=1)

# Calculate the intermediate Shipping Price by summing Shipping Price and Additional Charges
updated_sku_data['Intermediate Shipping Price'] = updated_sku_data['Shipping Price Netherlands'] + updated_sku_data['Additional Charges']

# Apply the fuel surcharge of 17%
netherlands_fuel_surcharge = 0.17
updated_sku_data['Shipping Price with Fuel Surcharge'] = updated_sku_data['Intermediate Shipping Price'] * (1 + netherlands_fuel_surcharge)

# Apply the VAT of 21%
vat = 0.21
updated_sku_data['Final Shipping Price'] = updated_sku_data['Shipping Price with Fuel Surcharge'] * (1 + vat)



In [16]:
# Display the first few rows of the final dataset
updated_sku_data

,Variant SKU,Status,Variant ID,Variant Weight,Variant Weight Unit,Variant Price,Variant Cost,Variant Metafield: custom.height [single_line_text_field],Variant Metafield: custom.width [single_line_text_field],Variant Metafield: custom.length [single_line_text_field],...,Exceed Width,Exceed Height,Exceed Girth,Weight Type,Base Shipment,Shipping Price Netherlands,Additional Charges,Intermediate Shipping Price,Shipping Price with Fuel Surcharge,Final Shipping Price
0,TR8PRO-R2R-BUN2-EU,Active,42514735399106,126.6000,kg,4671.00,NaN,0.0,0.0,0.0,...,1,0,1,Volumetric,186.0,100.68,18,118.68,138.8556,168.015276
1,TR8PRO-R2R-BUN1-EU,Active,42514407063746,126.6000,kg,3901.00,NaN,0.0,0.0,0.0,...,0,0,1,Volumetric,144.0,78.00,18,96.00,112.3200,135.907200
2,TR80L-1-TK,Active,47223492411737,110.0000,kg,8478.00,NaN,0.0,0.0,0.0,...,0,0,1,Volumetric,117.0,63.42,18,81.42,95.2614,115.266294
3,TR8PRO-1-TK-EU,Active,47223428612441,40.3209,kg,6241.00,NaN,0.0,0.0,0.0,...,0,0,1,Volumetric,111.0,60.18,18,78.18,91.4706,110.679426
4,SA-09 BUNDLE,Active,47582883807577,8.0000,kg,338.00,NaN,0.0,0.0,0.0,...,0,0,1,Volumetric,84.0,45.60,18,63.60,74.4120,90.038520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,DRKV08346,Draft,48338871779673,0.4000,kg,5.00,NaN,NaN,NaN,NaN,...,0,0,0,True Weight,0.4,6.83,0,6.83,7.9911,9.669231
616,TR8-08-B-PART1,Draft,48373033795929,0.0000,kg,30.00,NaN,NaN,NaN,NaN,...,0,0,0,True Weight,0.0,6.83,0,6.83,7.9911,9.669231
617,SP-24-BLUE-TRX,Draft,48522933436761,0.5000,kg,15.00,NaN,NaN,NaN,NaN,...,0,0,0,True Weight,0.5,6.83,0,6.83,7.9911,9.669231
618,HE-SPUBP20,Draft,49006423966041,0.0000,kg,92.23,NaN,NaN,NaN,NaN,...,0,0,0,True Weight,0.0,6.83,0,6.83,7.9911,9.669231


In [17]:
output_file_path = 'Updated_Shipping_with_new_conditions_netherlands_updated.xlsx'
updated_sku_data.to_excel(output_file_path, index=False)

output_file_path

'Updated_Shipping_with_new_conditions_netherlands_updated.xlsx'

# New International

In [3]:


# Rename the columns in fedex_data to 'Weight', 'Price', 'Belgium Price', 'Germany Price', 'France Price'
fedex_data = fedex_data.rename(columns={
    fedex_data.columns[0]: 'Weight',
    fedex_data.columns[2]: 'Netherlands Price',
    fedex_data.columns[5]: 'Belgium Price',
    fedex_data.columns[6]: 'Germany Price',
    fedex_data.columns[7]: 'France Price'
})[['Weight', 'Netherlands Price', 'Belgium Price', 'Germany Price', 'France Price']]

# Remove any row where 'Price' is NaN
fedex_data.dropna(subset=['Netherlands Price', 'Belgium Price', 'Germany Price', 'France Price'], inplace=True)

# Load the Updated SKU Data
updated_sku_data = new_updated_sku_data

# Function to calculate Base Shipment
def calculate_base_shipment(row):
    if row['Weight Type'] == 'Volumetric':
        return row['Variant Metafield: custom.volumetric_weight [single_line_text_field]']
    elif row['Weight Type'] == 'True Weight':
        return row['Variant Weight']
    else:
        return None

# Apply the function to create the Base Shipment column
updated_sku_data['Base Shipment'] = updated_sku_data.apply(calculate_base_shipment, axis=1)

# Function to find the closest price based on the base shipment weight for a specific country
def find_closest_price(base_shipment_weight, country_price_column):
    closest_row = fedex_data.iloc[(fedex_data['Weight'] - base_shipment_weight).abs().argsort()[:1]]
    return closest_row[country_price_column].values[0] if not closest_row.empty else None

# Apply the function to calculate the Shipping Price for each country
updated_sku_data['Shipping Price Netherlands'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Netherlands Price')
updated_sku_data['Shipping Price Belgium'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Belgium Price')
updated_sku_data['Shipping Price Germany'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Germany Price')
updated_sku_data['Shipping Price France'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='France Price')




In [4]:
# Function to calculate additional charges
def calculate_additional_charges(row):
    if row['Exceed Length Quantity'] == 1 or row['Exceed Weight Quantity'] == 1 or row['Exceed Width'] == 1 or row['Exceed Height'] == 1 or row['Exceed Girth'] == 1:
        return 18
    else:
        return 0

# Apply the function to create a new column for the additional charges
updated_sku_data['Additional Charges'] = updated_sku_data.apply(calculate_additional_charges, axis=1)

# Function to calculate the final shipping price for a specific country with a specified fuel surcharge
def calculate_final_shipping_price(shipping_price_column, fuel_surcharge):
    # Intermediate Shipping Price by summing Shipping Price and Additional Charges
    updated_sku_data['Intermediate Shipping Price'] = updated_sku_data[shipping_price_column] + updated_sku_data['Additional Charges']
    
    # Apply the specified fuel surcharge
    updated_sku_data['Shipping Price with Fuel Surcharge'] = updated_sku_data['Intermediate Shipping Price'] * (1 + fuel_surcharge)
    
    # Apply the VAT of 21%
    vat = 0.21
    return updated_sku_data['Shipping Price with Fuel Surcharge'] * (1 + vat)

# Calculate Final Shipping Price for Netherlands, Belgium, Germany, and France
updated_sku_data['Final Shipping Price Netherlands'] = calculate_final_shipping_price('Shipping Price Netherlands', fuel_surcharge=0.17)
updated_sku_data['Final Shipping Price Belgium'] = calculate_final_shipping_price('Shipping Price Belgium', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price Germany'] = calculate_final_shipping_price('Shipping Price Germany', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price France'] = calculate_final_shipping_price('Shipping Price France', fuel_surcharge=0.29)

# Display the first few rows of the final dataset
updated_sku_data.head

(          Variant SKU  Status      Variant ID  Variant Weight  \
 0  TR8PRO-R2R-BUN2-EU  Active  42514735399106        126.6000   
 1  TR8PRO-R2R-BUN1-EU  Active  42514407063746        126.6000   
 2          TR80L-1-TK  Active  47223492411737        110.0000   
 3      TR8PRO-1-TK-EU  Active  47223428612441         40.3209   
 4        SA-09 BUNDLE  Active  47582883807577          8.0000   
 
   Variant Weight Unit  Variant Price  Variant Cost  \
 0                  kg         4671.0           NaN   
 1                  kg         3901.0           NaN   
 2                  kg         8478.0           NaN   
 3                  kg         6241.0           NaN   
 4                  kg          338.0           NaN   
 
    Variant Metafield: custom.height [single_line_text_field]  \
 0                                                0.0           
 1                                                0.0           
 2                                                0.0           
 3        

In [5]:
output_file_path = 'International_BGF__Shipping_with_new_conditions_netherlands_updated.xlsx'
updated_sku_data.to_excel(output_file_path, index=False)

output_file_path

'International_BGF__Shipping_with_new_conditions_netherlands_updated.xlsx'

### EUROPE REST AND INTERNATIONAL INCLUDED

In [9]:
import pandas as pd

# Rename the columns in fedex_data to 'Weight', 'Price', 'Belgium Price', 'Germany Price', 'France Price', 'Europe(REST) Price', and 'International Price'
fedex_data = fedex_data.rename(columns={
    fedex_data.columns[0]: 'Weight',
    fedex_data.columns[2]: 'Netherlands Price',
    fedex_data.columns[5]: 'Belgium Price',
    fedex_data.columns[6]: 'Germany Price',
    fedex_data.columns[7]: 'France Price',
    fedex_data.columns[8]: 'Europe(REST) Price',
    fedex_data.columns[9]: 'International Price'
})[['Weight', 'Netherlands Price', 'Belgium Price', 'Germany Price', 'France Price', 'Europe(REST) Price', 'International Price']]

# Remove any row where 'Price' is NaN
fedex_data.dropna(subset=['Netherlands Price', 'Belgium Price', 'Germany Price', 'France Price', 'Europe(REST) Price', 'International Price'], inplace=True)

# Load the Updated SKU Data
updated_sku_data = new_updated_sku_data

# Function to calculate Base Shipment
def calculate_base_shipment(row):
    if row['Weight Type'] == 'Volumetric':
        return row['Variant Metafield: custom.volumetric_weight [single_line_text_field]']
    elif row['Weight Type'] == 'True Weight':
        return row['Variant Weight']
    else:
        return None

# Apply the function to create the Base Shipment column
updated_sku_data['Base Shipment'] = updated_sku_data.apply(calculate_base_shipment, axis=1)

# Function to find the closest price based on the base shipment weight for a specific country
def find_closest_price(base_shipment_weight, country_price_column):
    closest_row = fedex_data.iloc[(fedex_data['Weight'] - base_shipment_weight).abs().argsort()[:1]]
    return closest_row[country_price_column].values[0] if not closest_row.empty else None

# Apply the function to calculate the Shipping Price for each country/region
updated_sku_data['Shipping Price Netherlands'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Netherlands Price')
updated_sku_data['Shipping Price Belgium'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Belgium Price')
updated_sku_data['Shipping Price Germany'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Germany Price')
updated_sku_data['Shipping Price France'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='France Price')
updated_sku_data['Shipping Price Europe(REST)'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='Europe(REST) Price')
updated_sku_data['Shipping Price International'] = updated_sku_data['Base Shipment'].apply(find_closest_price, country_price_column='International Price')

# Function to calculate additional charges
def calculate_additional_charges(row):
    if row['Exceed Length Quantity'] == 1 or row['Exceed Weight Quantity'] == 1 or row['Exceed Width'] == 1 or row['Exceed Height'] == 1 or row['Exceed Girth'] == 1:
        return 18
    else:
        return 0

# Apply the function to create a new column for the additional charges
updated_sku_data['Additional Charges'] = updated_sku_data.apply(calculate_additional_charges, axis=1)

# Function to calculate the final shipping price for a specific country with a specified fuel surcharge and VAT option
def calculate_final_shipping_price(shipping_price_column, fuel_surcharge, apply_vat=True):
    # Intermediate Shipping Price by summing Shipping Price and Additional Charges
    updated_sku_data['Intermediate Shipping Price'] = updated_sku_data[shipping_price_column] + updated_sku_data['Additional Charges']
    
    # Apply the specified fuel surcharge
    updated_sku_data['Shipping Price with Fuel Surcharge'] = updated_sku_data['Intermediate Shipping Price'] * (1 + fuel_surcharge)
    
    # Apply VAT if required
    if apply_vat:
        vat = 0.21
        return updated_sku_data['Shipping Price with Fuel Surcharge'] * (1 + vat)
    else:
        return updated_sku_data['Shipping Price with Fuel Surcharge']

# Calculate Final Shipping Price for all regions/countries
updated_sku_data['Final Shipping Price Netherlands'] = calculate_final_shipping_price('Shipping Price Netherlands', fuel_surcharge=0.17)
updated_sku_data['Final Shipping Price Belgium'] = calculate_final_shipping_price('Shipping Price Belgium', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price Germany'] = calculate_final_shipping_price('Shipping Price Germany', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price France'] = calculate_final_shipping_price('Shipping Price France', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price Europe(REST)'] = calculate_final_shipping_price('Shipping Price Europe(REST)', fuel_surcharge=0.29)
updated_sku_data['Final Shipping Price International'] = calculate_final_shipping_price('Shipping Price International', fuel_surcharge=0.29, apply_vat=False)

# Display the first few rows of the final dataset
updated_sku_data.head(), fedex_data.head()

(          Variant SKU  Status      Variant ID  Variant Weight  \
 0  TR8PRO-R2R-BUN2-EU  Active  42514735399106        126.6000   
 1  TR8PRO-R2R-BUN1-EU  Active  42514407063746        126.6000   
 2          TR80L-1-TK  Active  47223492411737        110.0000   
 3      TR8PRO-1-TK-EU  Active  47223428612441         40.3209   
 4        SA-09 BUNDLE  Active  47582883807577          8.0000   
 
   Variant Weight Unit  Variant Price  Variant Cost  \
 0                  kg         4671.0           NaN   
 1                  kg         3901.0           NaN   
 2                  kg         8478.0           NaN   
 3                  kg         6241.0           NaN   
 4                  kg          338.0           NaN   
 
    Variant Metafield: custom.height [single_line_text_field]  \
 0                                                0.0           
 1                                                0.0           
 2                                                0.0           
 3        

In [10]:
output_file_path = 'Europe(REST)_International_BGF__Shipping_with_new_conditions_netherlands_updated.xlsx'
updated_sku_data.to_excel(output_file_path, index=False)

output_file_path

'Europe(REST)_International_BGF__Shipping_with_new_conditions_netherlands_updated.xlsx'

# Add International

In [14]:
import pandas as pd

# Load the necessary data
international_shipping_file_path = 'Olcay fedex int^L.xlsx'
updated_sku_data_final_path = 'Shipping_with_new_conditions_netherlands.xlsx'

# Load the international shipping data
international_shipping_data = pd.read_excel(international_shipping_file_path, sheet_name='Sheet1')

# Load the Updated SKU Data
updated_sku_data = pd.read_excel(updated_sku_data_final_path)

# Function to calculate Base Shipment
def calculate_base_shipment(row):
    if row['Weight Type'] == 'Volumetric':
        return row['Variant Metafield: custom.volumetric_weight [single_line_text_field]']
    elif row['Weight Type'] == 'True Weight':
        return row['Variant Weight']
    else:
        return None

# Apply the function to create the Base Shipment column
updated_sku_data['Base Shipment'] = updated_sku_data.apply(calculate_base_shipment, axis=1)

# Function to find the closest price based on the base shipment weight for a specific country
def find_closest_price_in_country(base_shipment_weight, country_col):
    closest_row = international_shipping_data.iloc[(international_shipping_data['Kgs'] - base_shipment_weight).abs().argsort()[:1]]
    if closest_row.empty or country_col not in closest_row.columns:
        return None
    return closest_row[country_col].values[0]

# List of specific countries to create columns for, EXCLUDING Netherlands
specific_countries = ['Germany', 'France', 'Belgium', 'Italy', 'Spain']

# Calculate shipping prices for specific countries
for country in specific_countries:
    col_name = f'Shipping Price {country}'
    updated_sku_data[col_name] = updated_sku_data['Base Shipment'].apply(lambda weight: find_closest_price_in_country(weight, country))

# Group other European countries (excluding Norway, Switzerland, and the United Kingdom) under "Europe - Rest"
europe_rest_countries = ['Austria', 'Denmark', 'Ireland', 'Luxemburg', 'Sweden']
updated_sku_data['Shipping Price Europe - Rest'] = updated_sku_data['Base Shipment'].apply(
    lambda weight: sum(find_closest_price_in_country(weight, country) for country in europe_rest_countries if find_closest_price_in_country(weight, country) is not None) / len(europe_rest_countries)
)

# Separate columns for United Kingdom and International (Norway and Switzerland)
updated_sku_data['Shipping Price United Kingdom'] = updated_sku_data['Base Shipment'].apply(lambda weight: find_closest_price_in_country(weight, 'United Kingdom'))
updated_sku_data['Shipping Price International'] = updated_sku_data['Base Shipment'].apply(lambda weight: sum(find_closest_price_in_country(weight, country) for country in ['Norway', 'Switzerland']) / 2)

# Function to calculate additional charges based on Exceed Length and Weight Quantity columns
def calculate_additional_charges(row):
    if row['Exceed Length Quantity'] == 1:
        return 18
    elif row['Exceed Weight Quantity'] == 1:
        return 18
    else:
        return 0

# Apply the function to create a new column for the additional charges
updated_sku_data['Additional Charges'] = updated_sku_data.apply(calculate_additional_charges, axis=1)

# Calculate the Final Shipping Price for each country by summing Shipping Price and Additional Charges
for country in specific_countries + ['Europe - Rest', 'United Kingdom', 'International']:
    col_name = f'Shipping Price {country}'
    final_price_col = f'Final Shipping Price {country}'
    updated_sku_data[final_price_col] = updated_sku_data[col_name] + updated_sku_data['Additional Charges']

# Save the updated dataset to the same file path, which will overwrite it
updated_sku_data.to_excel(updated_sku_data_final_path, index=False)
